In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from collections import defaultdict
import math
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random


class BERT4RecDataset(Dataset):
    """Датасет для BERT4Rec"""

    def __init__(self, sequences, max_len, mask_prob=0.15, num_items=None, mask_token=0):
        """
        Args:
            sequences: список последовательностей взаимодействий
            max_len: максимальная длина последовательности
            mask_prob: вероятность маскирования
            num_items: общее количество предметов
            mask_token: токен для маскирования
        """
        self.sequences = sequences
        self.max_len = max_len
        self.mask_prob = mask_prob
        self.num_items = num_items
        self.mask_token = mask_token

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]

        # Обрезка или дополнение последовательности
        if len(sequence) > self.max_len:
            sequence = sequence[-self.max_len:]
        else:
            sequence = [0] * (self.max_len - len(sequence)) + sequence

        # Создание маскированных версий
        masked_sequence = sequence.copy()
        labels = [0] * self.max_len  # 0 для игнорирования
        mask = [0] * self.max_len

        for i in range(self.max_len):
            if sequence[i] != 0:  # Не padding
                if random.random() < self.mask_prob:
                    prob = random.random()
                    if prob < 0.8:
                        # 80%: заменить на [MASK]
                        masked_sequence[i] = self.mask_token
                    elif prob < 0.9:
                        # 10%: заменить случайным предметом
                        # Убедимся, что случайный предмет в допустимом диапазоне
                        masked_sequence[i] = random.randint(2, self.num_items + 1)
                    # 10%: оставить как есть
                    labels[i] = sequence[i]
                    mask[i] = 1

        return {
            'input_seq': torch.LongTensor(masked_sequence),
            'labels': torch.LongTensor(labels),
            'mask': torch.LongTensor(mask),
            'original_seq': torch.LongTensor(sequence)
        }


class PositionalEncoding(nn.Module):
    """Позиционное кодирование для Transformer"""

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                           (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]


class BERT4Rec(nn.Module):
    """Модель BERT4Rec для рекомендательных систем"""

    def __init__(self, num_items, max_len, d_model=256, nhead=4,
                 num_layers=2, dropout=0.1):
        """
        Args:
            num_items: количество уникальных предметов
            max_len: максимальная длина последовательности
            d_model: размерность эмбеддингов
            nhead: количество head'ов в multi-head attention
            num_layers: количество слоев Transformer
            dropout: вероятность dropout
        """
        super(BERT4Rec, self).__init__()

        self.num_items = num_items
        self.max_len = max_len
        self.d_model = d_model

        # Важно: +2 для padding (0) и mask (1) токенов
        # Выходной слой должен предсказывать num_items + 1 (без учета padding)
        self.item_emb = nn.Embedding(num_items + 2, d_model, padding_idx=0)

        # Positional encoding
        self.pos_encoder = PositionalEncoding(d_model, max_len)

        # Transformer layers
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=d_model * 4,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )

        # Layer normalization
        self.layer_norm = nn.LayerNorm(d_model)

        # Output layer: предсказываем num_items + 1 (игнорируя padding token 0)
        self.output = nn.Linear(d_model, num_items + 1)

        # Dropout
        self.dropout = nn.Dropout(dropout)

        # Инициализация весов
        self._init_weights()

    def _init_weights(self):
        """Инициализация весов"""
        initrange = 0.1
        self.item_emb.weight.data.uniform_(-initrange, initrange)
        self.output.bias.data.zero_()
        self.output.weight.data.uniform_(-initrange, initrange)

    def forward(self, input_seq):
        """
        Args:
            input_seq: тензор формы [batch_size, seq_len]
        Returns:
            logits: тензор формы [batch_size, seq_len, num_items+1]
        """
        # Получаем эмбеддинги
        item_emb = self.item_emb(input_seq)  # [batch_size, seq_len, d_model]

        # Добавляем позиционное кодирование
        seq_emb = self.pos_encoder(item_emb)

        # Применяем dropout
        seq_emb = self.dropout(seq_emb)

        # Создаем маску для padding
        padding_mask = (input_seq == 0)

        # Пропускаем через Transformer
        transformer_output = self.transformer(
            seq_emb,
            src_key_padding_mask=padding_mask
        )

        # Layer normalization
        transformer_output = self.layer_norm(transformer_output)

        # Output layer
        logits = self.output(transformer_output)

        return logits

    def predict_next(self, input_seq, k=10):
        """
        Предсказание следующего предмета в последовательности

        Args:
            input_seq: последовательность предметов
            k: количество предметов для рекомендации
        Returns:
            top_k_items: топ-k рекомендованных предметов
            top_k_probs: их вероятности
        """
        self.eval()
        with torch.no_grad():
            # Подготовка входных данных
            if len(input_seq) > self.max_len:
                input_seq = input_seq[-self.max_len:]
            else:
                input_seq = [0] * (self.max_len - len(input_seq)) + input_seq

            input_tensor = torch.LongTensor(input_seq).unsqueeze(0).to(next(self.parameters()).device)

            # Получаем предсказания
            logits = self.forward(input_tensor)

            # Берем предсказание для последней позиции
            last_logits = logits[0, -1, :]
            probs = F.softmax(last_logits, dim=-1)

            # Получаем топ-k предметов
            # +1 потому что output layer не включает padding token (0)
            top_k_probs, top_k_indices = torch.topk(probs, k)

            return top_k_indices.cpu().numpy(), top_k_probs.cpu().numpy()


class BERT4RecTrainer:
    """Тренер для модели BERT4Rec"""

    def __init__(self, model, train_loader, val_loader, device='cuda'):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
        # Игнорируем padding token (0) в loss
        self.criterion = nn.CrossEntropyLoss(ignore_index=0)

    def train(self, num_epochs=50, lr=0.001):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

        best_val_loss = float('inf')

        for epoch in range(num_epochs):
            # Training
            self.model.train()
            train_loss = 0
            train_steps = 0

            pbar = tqdm(self.train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
            for batch in pbar:
                input_seq = batch['input_seq'].to(self.device)
                labels = batch['labels'].to(self.device)
                mask = batch['mask'].to(self.device)

                optimizer.zero_grad()

                # Forward pass
                logits = self.model(input_seq)

                # Вычисляем loss только для замаскированных позиций
                batch_size, seq_len, num_preds = logits.shape
                logits = logits.view(-1, num_preds)
                labels = labels.view(-1)
                mask = mask.view(-1)

                # Применяем маску
                masked_logits = logits[mask == 1]
                masked_labels = labels[mask == 1]

                if len(masked_labels) > 0:
                    # Важно: уменьшаем labels на 1, потому что выходной слой
                    # предсказывает num_items+1, начиная с 0
                    # и padding token (0) игнорируется
                    loss_labels = masked_labels - 1

                    # Проверяем границы
                    if (loss_labels >= 0).all() and (loss_labels < num_preds).all():
                        loss = self.criterion(masked_logits, loss_labels)
                        loss.backward()
                        optimizer.step()

                        train_loss += loss.item()
                        train_steps += 1

                pbar.set_postfix({'loss': loss.item() if 'loss' in locals() else 0})

            # Validation
            val_loss, val_metrics = self.evaluate()

            # Сохраняем лучшую модель
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(self.model.state_dict(), 'bert4rec_best.pth')

            print(f'Epoch {epoch+1}: Train Loss: {train_loss/train_steps:.4f}, '
                  f'Val Loss: {val_loss:.4f}, '
                  f'Val Metrics: {val_metrics}')

            scheduler.step()

    def evaluate(self):
        self.model.eval()
        val_loss = 0
        val_steps = 0

        # Метрики
        hits_at_10 = 0
        ndcg_at_10 = 0
        total_samples = 0

        with torch.no_grad():
            for batch in self.val_loader:
                input_seq = batch['input_seq'].to(self.device)
                labels = batch['labels'].to(self.device)
                mask = batch['mask'].to(self.device)
                original_seq = batch['original_seq'].to(self.device)

                # Forward pass
                logits = self.model(input_seq)

                # Вычисляем loss
                batch_size, seq_len, num_preds = logits.shape
                logits = logits.view(-1, num_preds)
                labels = labels.view(-1)
                mask = mask.view(-1)

                masked_logits = logits[mask == 1]
                masked_labels = labels[mask == 1]

                if len(masked_labels) > 0:
                    loss_labels = masked_labels - 1
                    if (loss_labels >= 0).all() and (loss_labels < num_preds).all():
                        loss = self.criterion(masked_logits, loss_labels)
                        val_loss += loss.item()
                        val_steps += 1

                # Вычисляем метрики для последней позиции
                for i in range(batch_size):
                    # Берем последний ненулевой элемент из оригинальной последовательности
                    seq = original_seq[i].cpu().numpy()
                    non_zero_idx = np.where(seq != 0)[0]
                    if len(non_zero_idx) > 0:
                        last_item = seq[non_zero_idx[-1]]

                        # Получаем предсказания для последней позиции
                        last_logits = logits[i * seq_len + (seq_len - 1):(i + 1) * seq_len]
                        probs = F.softmax(last_logits, dim=-1)

                        # Топ-10 предсказаний
                        top_10 = torch.topk(probs, min(10, len(probs))).indices.cpu().numpy()

                        # Преобразуем обратно к исходным индексам (+1)
                        top_10_items = top_10 + 1

                        # Hit Rate @ 10
                        if last_item in top_10_items:
                            hits_at_10 += 1

                            # NDCG @ 10
                            rank = np.where(top_10_items == last_item)[0][0] + 1
                            ndcg_at_10 += 1 / np.log2(rank + 1)

                        total_samples += 1

        metrics = {
            'HR@10': hits_at_10 / max(total_samples, 1),
            'NDCG@10': ndcg_at_10 / max(total_samples, 1)
        }

        return val_loss / max(val_steps, 1), metrics


def prepare_data(ratings, min_interactions=5, test_size=0.2):
    """
    Подготовка данных для BERT4Rec

    Args:
        ratings: DataFrame с колонками ['user_id', 'item_id', 'timestamp']
        min_interactions: минимальное количество взаимодействий для пользователя
        test_size: доля тестовых данных
    """
    # Фильтрация пользователей
    user_counts = ratings['user_id'].value_counts()
    valid_users = user_counts[user_counts >= min_interactions].index
    ratings = ratings[ratings['user_id'].isin(valid_users)]

    # Сортировка по времени
    ratings = ratings.sort_values(['user_id', 'timestamp'])

    # Создание последовательностей
    sequences = []
    for user_id, group in ratings.groupby('user_id'):
        seq = group['item_id'].tolist()
        sequences.append(seq)

    # Разделение на train/val
    train_seq, val_seq = train_test_split(sequences, test_size=test_size, random_state=42)

    # Создание словаря предметов
    all_items = set()
    for seq in sequences:
        all_items.update(seq)
    num_items = len(all_items)

    # Перенумерация предметов:
    # 0 - padding token
    # 1 - mask token
    # 2+ - реальные предметы
    item_to_idx = {item: i+2 for i, item in enumerate(all_items)}
    idx_to_item = {i+2: item for i, item in enumerate(all_items)}

    # Преобразование последовательностей
    train_seq_idx = [[item_to_idx[item] for item in seq] for seq in train_seq]
    val_seq_idx = [[item_to_idx[item] for item in seq] for seq in val_seq]

    return train_seq_idx, val_seq_idx, num_items, idx_to_item


def create_synthetic_data(num_users=1000, num_items=500, num_interactions=20000):
    """Создание синтетических данных для демонстрации"""
    np.random.seed(42)

    # Генерация данных
    ratings_data = {
        'user_id': np.random.randint(1, num_users+1, num_interactions),
        'item_id': np.random.randint(1, num_items+1, num_interactions),
        'timestamp': np.arange(num_interactions)
    }

    # Добавим некоторые паттерны для реалистичности
    for i in range(num_interactions):
        # Некоторые пользователи предпочитают определенные предметы
        if ratings_data['user_id'][i] % 10 == 0:
            ratings_data['item_id'][i] = np.random.randint(1, 51)

    return pd.DataFrame(ratings_data)


def main():
    """Пример использования BERT4Rec"""

    # Параметры
    MAX_LEN = 20  # Уменьшим для быстрого тестирования
    BATCH_SIZE = 32
    NUM_EPOCHS = 5  # Уменьшим для демонстрации
    D_MODEL = 64  # Уменьшим размерность
    NHEAD = 2  # Уменьшим количество head'ов
    NUM_LAYERS = 1  # Уменьшим количество слоев
    DROPOUT = 0.1
    MASK_PROB = 0.2
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

    print(f"Используется устройство: {DEVICE}")

    # Создание синтетических данных
    print("Создание синтетических данных...")
    ratings_df = create_synthetic_data(
        num_users=200,  # Уменьшим для быстрого тестирования
        num_items=100,
        num_interactions=5000
    )

    # Подготовка данных
    print("Подготовка данных...")
    train_seq, val_seq, num_items, idx_to_item = prepare_data(
        ratings_df, min_interactions=3, test_size=0.1
    )

    print(f"Количество train последовательностей: {len(train_seq)}")
    print(f"Количество val последовательностей: {len(val_seq)}")
    print(f"Количество уникальных предметов: {num_items}")
    print(f"Длина последовательности: {MAX_LEN}")

    # Создание датасетов и загрузчиков данных
    train_dataset = BERT4RecDataset(
        train_seq,
        max_len=MAX_LEN,
        mask_prob=MASK_PROB,
        num_items=num_items,
        mask_token=1  # Токен для маскирования
    )

    val_dataset = BERT4RecDataset(
        val_seq,
        max_len=MAX_LEN,
        mask_prob=MASK_PROB,
        num_items=num_items,
        mask_token=1
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=0
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=0
    )

    # Создание модели
    print("Создание модели...")
    model = BERT4Rec(
        num_items=num_items,
        max_len=MAX_LEN,
        d_model=D_MODEL,
        nhead=NHEAD,
        num_layers=NUM_LAYERS,
        dropout=DROPOUT
    )

    print(f"Модель создана. Параметров: {sum(p.numel() for p in model.parameters()):,}")

    # Обучение
    print("Начало обучения...")
    trainer = BERT4RecTrainer(model, train_loader, val_loader, device=DEVICE)
    trainer.train(num_epochs=NUM_EPOCHS, lr=0.001)

    # Пример предсказания
    print("\nПример предсказания...")
    if len(train_seq) > 0:
        test_sequence = train_seq[0][-5:]  # Берем последние 5 взаимодействий
        print(f"История пользователя: {[idx_to_item[idx] for idx in test_sequence if idx != 0]}")

        try:
            model.load_state_dict(torch.load('bert4rec_best.pth', map_location=DEVICE))
            top_items, top_probs = model.predict_next(test_sequence, k=5)
            print("Топ-5 рекомендаций:")
            for item_idx, prob in zip(top_items, top_probs):
                # Преобразуем обратно к исходным индексам
                if item_idx + 1 in idx_to_item:
                    print(f"  Предмет {idx_to_item[item_idx + 1]}: вероятность {prob:.4f}")
                else:
                    print(f"  Предмет {item_idx + 1}: вероятность {prob:.4f}")
        except:
            print("Не удалось загрузить модель для предсказания")

    print("\nОбучение завершено!")


if __name__ == "__main__":
    main()

Используется устройство: cpu
Создание синтетических данных...
Подготовка данных...
Количество train последовательностей: 180
Количество val последовательностей: 20
Количество уникальных предметов: 100
Длина последовательности: 20
Создание модели...
Модель создана. Параметров: 63,205
Начало обучения...


Epoch 1/5: 100%|██████████| 6/6 [00:00<00:00, 13.60it/s, loss=4.73]


Epoch 1: Train Loss: 4.7000, Val Loss: 4.6078, Val Metrics: {'HR@10': 0.05, 'NDCG@10': np.float64(0.05)}


Epoch 2/5: 100%|██████████| 6/6 [00:00<00:00, 17.08it/s, loss=4.78]


Epoch 2: Train Loss: 4.6939, Val Loss: 4.6132, Val Metrics: {'HR@10': 0.05, 'NDCG@10': np.float64(0.05)}


...

Epoch 3: Train Loss: 4.6562, Val Loss: 4.6829, Val Metrics: {'HR@10': 0.05, 'NDCG@10': np.float64(0.05)}


Epoch 4/5: 100%|██████████| 6/6 [00:00<00:00, 19.30it/s, loss=4.68]


Epoch 4: Train Loss: 4.6486, Val Loss: 4.6575, Val Metrics: {'HR@10': 0.05, 'NDCG@10': np.float64(0.05)}


Epoch 5/5: 100%|██████████| 6/6 [00:00<00:00, 24.48it/s, loss=4.62]


Epoch 5: Train Loss: 4.6287, Val Loss: 4.6720, Val Metrics: {'HR@10': 0.05, 'NDCG@10': np.float64(0.05)}

Пример предсказания...
История пользователя: [14, 3, 80, 14, 92]
Топ-5 рекомендаций:
  Предмет 42: вероятность 0.0252
  Предмет 22: вероятность 0.0226
  Предмет 7: вероятность 0.0216
  Предмет 18: вероятность 0.0202
  Предмет 58: вероятность 0.0187

Обучение завершено!
